In [1]:
###importing packages
import newspaper
from newspaper import Article
from newspaper import fulltext
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import time
import selenium 
from selenium import webdriver
from selenium.webdriver.common.proxy import Proxy, ProxyType
from selenium.webdriver.chrome.options import Options

In [249]:
###establishing api key
NYT_KEY = %env NYT_KEY 

In [183]:
### creating lists of all parameters to loop through
begin_date = [20180701, 20190701, 20200701]
end_date = [20180930, 20190930, 20200930]
states = ["Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado","Connecticut", "Delaware","Florida",
          "Georgia", "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa", "Kansas", "Kentucky", "Louisiana", "Maine",
          "Maryland", "Massachusetts", "Michigan", "Minnesota", "Mississippi", "Missouri", "Montana", "Nebraska",
          "Nevada", "New Hampshire", "New Jersey", "New Mexico", "New York", "North Carolina", "North Dakota", 
          "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Rhode Island", "South Carolina", "South Dakota", 
          "Tennessee", "Texas", "Utah", "Vermont", "Virginia", "Washington", "West Virginia", "Wisconsin", 
          "Wyoming"]

In [250]:
###creating function to pull NYT urls for climate articles
def api_call(i, state, ed, bd):
    try:
        params = {'api-key':NYT_KEY, 'q': 'climate change',
                  'fq': ['type_of_material = Article','source = The New York Times', 'glocations = %s' % (state)], 
                  'fl' : ['abstract', 'web_url', 'headline', 'pub_date', 'byline', 'keywords', 'word_count'],
                  'begin_date': bd, 'end_date': ed, 'page': i}
        response = requests.get('https://api.nytimes.com/svc/search/v2/articlesearch.json', params = params)
        nyt_json = response.json()
        nyt_json_df = pd.DataFrame(nyt_json['response']['docs'])
        return nyt_json_df
    except KeyError:
        error = nyt_json['fault']
        return error['faultstring']

In [163]:
###seeing how many hits there are
def hit_call(state, bd, ed):
    try:
        params = {'api-key':NYT_KEY, 'q': 'climate change',
                  'fq': ['type_of_material = Article','source = The New York Times', 'glocations = %s' % (state)], 
                  'fl' : ['abstract', 'web_url', 'headline', 'pub_date', 'byline', 'keywords', 'word_count'],
                  'begin_date': bd, 'end_date': ed}
        response = requests.get('https://api.nytimes.com/svc/search/v2/articlesearch.json', params = params)
        nyt_json = response.json()
        nyt_json_df = pd.DataFrame(nyt_json['response']['meta'], index = [bd])
        return nyt_json_df
    except KeyError:
        return nyt_json

In [109]:
hit_dict = pd.concat(hit_records)
hit_dict

ValueError: No objects to concatenate

In [78]:
hit_records = {}
for state in states:
    state_dict = []
    for dates in range(len(begin_date)):
        time.sleep(12)
        if hit_call(state = state, bd = begin_date[dates], ed = end_date[dates])
        state_dict.append(hit_call(state = state, bd = begin_date[dates], ed = end_date[dates]))
    hit_records[state] = pd.concat(state_dict)
   

KeyboardInterrupt: 

In [262]:
article_records = {}

In [258]:
state_dict

[]

In [263]:
###pulling articles for remaining pages of api call 
for state in states:
    state_dict = []
    for dates in range(len(begin_date)):
        for i in range(1,3):
            time.sleep(12)
            call = api_call(state = state, bd = begin_date[dates], ed = end_date[dates], i = i)
            if type(call) == str:
                print('Reached Max Quota')
            else:
                state_dict.append(call)
    article_records[state] = pd.concat(state_dict, ignore_index = True)
   

In [289]:
merged_df = pd.concat(article_records.values(), keys = list(article_records.keys()))

In [287]:
for i in range(len(test['web_url'])):
    print(test['web_url'][i])

https://www.nytimes.com/2018/08/31/podcasts/the-daily/climate-change-losing-earth.html
https://www.nytimes.com/2018/08/29/science/lake-superior-algae-toxic.html
https://www.nytimes.com/2018/09/04/climate/roger-federer-loss-heat.html
https://www.nytimes.com/2018/08/22/t-magazine/climate-change-art.html
https://www.nytimes.com/2018/09/26/climate/climate-week-and-photo-stories.html
https://www.nytimes.com/2018/08/30/arts/design/climate-museum-signs.html
https://www.nytimes.com/2018/09/21/climate/climate-week-nyc-2018.html
https://www.nytimes.com/2018/09/30/world/africa/senegal-baobabs-climate-change.html
https://www.nytimes.com/2018/09/26/opinion/amazon-climate-change-deforestation.html
https://www.nytimes.com/2018/09/27/nyregion/new-york-flooding.html
https://www.nytimes.com/interactive/2018/09/25/climate/scotland-orkney-islands-sea-level.html
https://www.nytimes.com/2018/09/27/climate/offshore-drilling-safety-deepwater-horizon.html
https://www.nytimes.com/2018/09/30/climate/epa-trump-me

In [291]:
###creating function to pull text from articles using urls pulled from nyt api
def article_download(pulled_url):
    article = Article(url = pulled_url, fetch_images = False, keep_article_html = True)
    article.download()
    article.parse()
    article.clean_top_node
    article_dict = {'text': article.text,
                   'html': article.html}
    return article_dict

In [288]:
text_df = []

In [292]:
###pulling out article text from every article url
for rows in range(len(merged_df['web_url'])):
    text = article_download(merged_df['web_url'][rows])['text']
    text_df.append(text)

In [295]:
###adding column within nyt_df containing text
merged_df['text'] = text_df

In [296]:
merged_df

abstract  \
Alabama 0   The U.S. had an opportunity to solve the clima...   
        1   Scientists cannot say for sure what causes alg...   
        2   Federer struggled in the nighttime heat and hu...   
        3   A dozen artistic responses to one of the great...   
        4   This week world leaders are meeting in New Yor...   
...                                                       ...   
Wyoming 55  Court losses are piling up for President Trump...   
        56  The effort to open the Alaskan wilderness area...   
        57  One after another, major fires exploded across...   
        58  Under international pressure to do more to add...   
        59  There’s a huge fire debt in the West that must...   

                                                      web_url  \
Alabama 0   https://www.nytimes.com/2018/08/31/podcasts/th...   
        1   https://www.nytimes.com/2018/08/29/science/lak...   
        2   https://www.nytimes.com/2018/09/04/climate/rog...   
        3   https://www.nytimes.com/2018/08/22/t-magazine/...   
        4   https://www.nytimes.com/2018/09/26/climate/cli...   
...                                                       ...   
Wyoming 55  https://www.nytimes.com/2020/09/23/climate/tru...   
        56  https://www.nytimes.com/2020/09/24/climate/ton...   
        57  https://www.nytimes.com/interactive/2020/09/24...   
        58  https://www.nytimes.com/2020/09/23/world/asia/...   
        59  https://www.nytimes.com/2020/09/23/climate/fir...   

                                                     headline  \
Alabama 0   {'main': 'When We Almost Stopped Climate Chang...   
        1   {'main': 'Algae Bloom in Lake Superior Raises ...   
        2   {'main': 'Roger Federer Is Tough to Beat. Glob...   
        3   {'main': '12 Artists On: Climate Change', 'kic...   
        4   {'main': 'Climate Week, and Telling Stories Wi...   
...                                                       ...   
Wyoming 55  {'main': 'Election and Supreme Court Fight Wil...   
        56  {'main': 'Trump Administration Releases Plan t...   
        57  {'main': 'Record Wildfires on the West Coast A...   
        58  {'main': 'China’s Pledge to Be Carbon Neutral ...   
        59  {'main': 'We’ll Have to Learn to Live With Smo...   

                                                     keywords  \
Alabama 0   [{'name': 'subject', 'value': 'Global Warming'...   
        1   [{'name': 'subject', 'value': 'Lakes', 'rank':...   
        2   [{'name': 'subject', 'value': 'Global Warming'...   
        3   [{'name': 'subject', 'value': 'Art', 'rank': 1...   
        4   [{'name': 'subject', 'value': 'Global Warming'...   
...                                                       ...   
Wyoming 55  [{'name': 'subject', 'value': 'Global Warming'...   
        56  [{'name': 'glocations', 'value': 'Tongass Nati...   
        57  [{'name': 'subject', 'value': 'Wildfires', 'ra...   
        58  [{'name': 'subject', 'value': 'Greenhouse Gas ...   
        59  [{'name': 'subject', 'value': 'Global Warming'...   

                            pub_date  \
Alabama 0   2018-08-31T09:59:54+0000   
        1   2018-08-29T17:34:54+0000   
        2   2018-09-04T22:16:22+0000   
        3   2018-08-22T15:02:32+0000   
        4   2018-09-26T17:49:12+0000   
...                              ...   
Wyoming 55  2020-09-23T09:00:22+0000   
        56  2020-09-25T00:41:22+0000   
        57  2020-09-24T04:53:06+0000   
        58  2020-09-23T10:56:13+0000   
        59  2020-09-23T14:21:32+0000   

                                                       byline  word_count  \
Alabama 0   {'original': None, 'person': [], 'organization...         228   
        1   {'original': 'By Christine Hauser', 'person': ...         951   
        2   {'original': 'By Kendra Pierre-Louis', 'person...         591   
        3   {'original': 'By Zoë Lescaze', 'person': [{'fi...        2304   
        4   {'original': None, 'person': [], 'organization...  

In [132]:
###if we want to pull full text using beautiful soup
soup_text = []
def SoupText(url):
    soup = BeautifulSoup(url, 'html.parser')
    text = soup.get_text()
    soup_text.append(text)
    soup_dict = {'soup':soup, 'text': text}
    return 

nyt_df['soup_text'] = soup_text





Why Smaller Storms Are Growing More Fearsome, More Often - The New York Times
  




















Skip to contentSkip to site indexSearch & Section NavigationSection NavigationSEARCHNew York Today’s PaperNew York|With Climate Change, Smaller Storms Are Growing More Fearsome, More Oftenhttps://www.nytimes.com/2023/09/30/nyregion/climate-change-flooding-storms.htmlShare full article384AdvertisementSKIP ADVERTISEMENTSupported bySKIP ADVERTISEMENTWith Climate Change, Smaller Storms Are Growing More Fearsome, More OftenThe Friday storm that produced vast flooding in New York City started out earlier in the week as an unremarkable — if unpredictable — weather system.Share full article384New Yorkers were taken by surprise Friday by the deluge of rain.Credit...Anna Watts for The New York TimesBy Hilary HowardSept. 30, 2023At first, it looked as if New York would simply be grazed by light rain on Friday.David Stark, a meteorologist with the National Oceanic and Atmospheric Administration

In [320]:
###trying other package https://github.com/susannapaoli/web-scraper-nyt
driver = webdriver.Chrome()

In [312]:
def get_body(url):
    driver.get(url)
    article_text = ''
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    paragraph = soup.find_all('p')
    for i in paragraph:
        a = i.get_text()
        if a != 'Advertisement' and a != 'Supported by' and a != 'Send any friend a story' and a != 'As a subscriber, you have 10 gift articles to give each month. Anyone can read what you share.' and not a.startswith("By"):
            article_text += a 
            article_text += " "
    time.sleep(8)
    return article_text

In [313]:
nyt_df['article_body'] = nyt_df.apply(lambda x: get_body(x.web_url), axis=1) 

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=119.0.6045.159)
  (Driver info: chromedriver=2.38.552518 (183d19265345f54ce39cbb94cf81ba5f15905011),platform=Mac OS X 10.16.0 x86_64)


In [297]:
###saving dataframe to csv file
merged_df.to_csv("/Users/lisayokocarle/Desktop/Columbia/Fall2023/NLP/FinalProject/qmss-nlp-climate/data/article_data.csv")